In [2]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"))

## Example Data

In [85]:
# block node
b1 = {
        "block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2",
        "block_height": "681545",
        "block_timestamp" : "2021-05-02 16:50:00",
        "p_block_hash": "00"
    }

b2 = {
        "block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395986",
        "block_height": "6815765",
        "block_timestamp" : "2021-05-02 17:00:00",
        "p_block_hash": "00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2"
    }

# transaction node 
t1 = {
    "txid": "cebc8ac65d27d9ae05aad5249e5b991e01d50b256118ea1722373a212814b81e",
    "block_date": "2021-05-02",
    "inDegree": 1,
    "outDegree": 4, 
    "inSum": 20.0,
    "outSum": 13.07
}

# address node 
a1 = {
    "address": "1EepjXgvWUoRyNvuLSAxjiqZ1QqKGDANLW",
    "inDegree": 1, 
    "outDegree": 1
}

# FOREACH Test
f = {
    "txid" : "id1",
    "addresses": [
        {
            "addr": "id_addr0" 
        },
        {
            "addr": "id_addr1" 
        },
        
    ], 
    "block":'asdlkjasdl'
    
}

## Transactions


In [86]:
def create_transaction(tx, txid, block_date, inDegree, outDegree, inSum, outSum):
    tx.run('MERGE (t:Transaction{txid: $txid, block_date: $block_date, inDegree: $inDegree, outDegree: $outDegree, inSum: $inSum, outSum: $outSum})',
           txid=txid, block_date=block_date, inDegree=inDegree, outDegree=outDegree, inSum=inSum, outSum=outSum)

In [87]:
t = t1
with driver.session() as session:
    session.write_transaction(create_transaction, 
                              t['txid'],
                              t['block_date'],
                              t['inDegree'],
                              t['outDegree'],
                              t['inSum'],
                              t['outSum'])

#### FOREACH Test

In [ ]:
def test_foreach(tx, txid, addresses):
    tx.run('MERGE (f:Tr{txid: $txid}) '
           'FOREACH (address in $addresses | MERGE '
           txid=txid, addresses=addresses)

## Blocks

#### Creating initial block for backreferencing

In [73]:
def create_initial_block(tx, initial_hash):
    tx.run('CREATE (b:Block{hash: $initial_hash})', initial_hash=initial_hash)

with driver.session() as session:
    session.write_transaction(create_initial_block, b1['p_block_hash'])

#### Function to create additional blocks

In [50]:
def create_block(tx, block_hash, block_height, block_timestamp):
    tx.run('MERGE (b:Block{hash: $block_hash, height: $block_height, timestamp: $block_timestamp})',
           block_hash=block_hash, block_height=block_height, block_timestamp=block_timestamp)

In [38]:
with driver.session() as session:
    session.write_transaction(create_block, b2['block_hash'], b2['block_height'], b2['block_timestamp'])

#### Block Creation and Back Referencing

The function creates the new block, matches the preceding block and creates the relationship between both.
<br>
Issue: breaks if preciding block not creates (not found(?))

In [78]:
def create_block(tx, block_hash, block_height, block_timestamp, p_block_hash):
    tx.run('MATCH (p:Block) WHERE p.hash = $p_block_hash ' 
           'CREATE (b:Block{hash: $block_hash, height: $block_height, timestamp: $block_timestamp}) '
           'MERGE (p)-[r:Precedes]->(b)',
           block_hash=block_hash, block_height=block_height, block_timestamp=block_timestamp, p_block_hash=p_block_hash)

In [79]:
block = b2
with driver.session() as session:
    session.write_transaction(create_block, 
                              block['block_hash'], 
                              block['block_height'], 
                              block['block_timestamp'],
                              block['p_block_hash'])
    